In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import itertools

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [107]:
df = pd.read_csv('novel-corona-virus-2019-dataset/covid_19_data.csv')
df.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [108]:
df.drop(['SNo', 'Last Update'], axis=1, inplace=True)
# df.ObservationDate = pd.to_datetime(df.ObservationDate, yearfirst=True)
df.head()

,ObservationDate,Province/State,Country/Region,Confirmed,Deaths,Recovered
0,01/22/2020,Anhui,Mainland China,1.0,0.0,0.0
1,01/22/2020,Beijing,Mainland China,14.0,0.0,0.0
2,01/22/2020,Chongqing,Mainland China,6.0,0.0,0.0
3,01/22/2020,Fujian,Mainland China,1.0,0.0,0.0
4,01/22/2020,Gansu,Mainland China,0.0,0.0,0.0


In [109]:
df = df.groupby(['ObservationDate', 'Country/Region']).sum().unstack(fill_value=0.0).stack().reset_index()
df.head()

,ObservationDate,Country/Region,Confirmed,Deaths,Recovered
0,01/22/2020,Azerbaijan,0.0,0.0,0.0
1,01/22/2020,"('St. Martin',)",0.0,0.0,0.0
2,01/22/2020,Afghanistan,0.0,0.0,0.0
3,01/22/2020,Albania,0.0,0.0,0.0
4,01/22/2020,Algeria,0.0,0.0,0.0


In [110]:
country_subset = ['US', 'Canada']
df_small = df[df['Country/Region'].isin(country_subset)].reset_index(drop=True)
df_small.head()

,ObservationDate,Country/Region,Confirmed,Deaths,Recovered
0,01/22/2020,Canada,0.0,0.0,0.0
1,01/22/2020,US,1.0,0.0,0.0
2,01/23/2020,Canada,0.0,0.0,0.0
3,01/23/2020,US,1.0,0.0,0.0
4,01/24/2020,Canada,0.0,0.0,0.0


In [111]:
df_small = pd.concat([df_small, *[df_small[['Confirmed', 'Deaths', 'Recovered']].shift(2*i) for i in range(1,4)]], axis=1).dropna().reset_index(drop=True)
df_small.head()

,ObservationDate,Country/Region,Confirmed,Deaths,Recovered,Confirmed,Deaths,Recovered,Confirmed,Deaths,Recovered,Confirmed,Deaths,Recovered
0,01/25/2020,Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01/25/2020,US,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,01/26/2020,Canada,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01/26/2020,US,5.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
4,01/27/2020,Canada,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


array([['01/25/2020', 'Canada', 0.0, ..., 0.0, 0.0, 0.0],
       ['01/25/2020', 'US', 2.0, ..., 1.0, 0.0, 0.0],
       ['01/26/2020', 'Canada', 1.0, ..., 0.0, 0.0, 0.0],
       ...,
       ['04/17/2020', 'US', 699706.0, ..., 607670.0, 25831.0, 47763.0],
       ['04/18/2020', 'Canada', 34356.0, ..., 28209.0, 1007.0, 8966.0],
       ['04/18/2020', 'US', 732197.0, ..., 636350.0, 28325.0, 52096.0]],
      dtype=object)

In [113]:
for date in df_small.ObservationDate.unique():
    temp = df_small[df_small.ObservationDate == date]
    G = nx.Graph()
    G.add_nodes_from(df_small['Country/Region'].tolist())
    G.add_edge('Canada', 'US')
    nx.write_gpickle(G, f'graphs/{date[0:2]+date[3:5]}.pkl')

In [100]:
str(date)

'2020-01-25T00:00:00.000000000'

In [0]:
df = df.drop(['SNo', 'Last Update'],axis = 1)
df = df.fillna('-1')
feature_vectors = pd.DataFrame()
feature_vectors['ObservationDate'] = df['ObservationDate'].unique()

In [0]:
nodes = np.array(df.groupby(['Province/State','Country/Region']).size().reset_index().rename(columns={0:'count'}))
nodes = nodes[:,0:2]

In [0]:
cases = np.zeros((86, 519, 3))
for idx, row in df.iterrows():
    row_idx = feature_vectors.index[feature_vectors['ObservationDate'] == row['ObservationDate']][0]
    if row['Province/State'] == -1:
        col_idx = np.where(nodes == row['Country/State'])[0][0]
    else:
        col_idx = np.where(nodes == row['Province/State'])[0][0]
    cases[row_idx, col_idx, 0] = row['Confirmed']
    cases[row_idx, col_idx, 1] = row['Deaths']
    cases[row_idx, col_idx, 2] = row['Recovered']

In [0]:
comb_nodes = np.array([])
for i in range(len(nodes)):
    if nodes[i][0] == '-1':
        comb_nodes = np.append(comb_nodes, nodes[i][1])
    else:
        comb_nodes = np.append(comb_nodes, nodes[i][0] + ', ' + nodes[i][1])

In [0]:
G = nx.Graph()
G.add_nodes_from(comb_nodes)
# All canadian + us border connections
G.add_edges_from([('Quebec, Canada', 'Montreal, QC, Canada'),
                  ('Quebec, Canada', 'Ontario, Canada'),
                  ('Quebec, Canada', 'Newfoundland and Labrador, Canada'),
                  ('Quebec, Canada', 'New Brunswick, Canada'),
                  ('Quebec, Canada', 'Maine, US'),
                  ('Quebec, Canada', 'Vermont, US'),
                  ('Quebec, Canada', 'New York, US'),
                  ('New Brunswick, Canada', 'Nova Scotia, Canada'),
                  ('New Brunswick, Canada', 'Maine, US'),
                  ('Prince Edward Island, Canada', 'Nova Scotia, Canada'),
                  ('Prince Edward Island, Canada', 'New Brunswick, Canada'),
                  ('Manitoba, Canada', 'Michigan, US'),
                  ('Manitoba, Canada', 'North Dakota, US'),
                  ('Saskatchewan, Canada', 'Manitoba, Canada'),
                  ('Saskatchewan, Canada', 'Alberta, Canada'),
                  ('Saskatchewan, Canada', 'Northwest Territories, Canada'),
                  ('Saskatchewan, Canada', 'Montana, US'),
                  ('Saskatchewan, Canada', 'North Dakota, US'),
                  ('Ontario, Canada', 'Toronto, ON, Canada'),
                  ('Ontario, Canada', 'London, ON, Canada'),
                  ('Ontario, Canada', 'Manitoba, Canada'),
                  ('Ontario, Canada', 'New York, US'),
                  ('Ontario, Canada', 'Michigan, US'),
                  ('Ontario, Canada', 'Minnesota, US'),
                  ('Alberta, Canada', 'Montana, US'),
                  ('Alberta, Canada', 'British Columbia, Canada'),
                  ('Alberta, Canada', 'Northwest Territories, Canada'),
                  ('Alberta, Canada', 'Calgary, Alberta, Canada'),
                  ('Alberta, Canada', 'Edmonton, Alberta, Canada'),
                  ('British Columbia, Canada', 'Washington, US'),
                  ('British Columbia, Canada', 'Montana, US'),
                  ('British Columbia, Canada', 'Idaho, US'),
                  ('British Columbia, Canada', 'Yukon, Canada'),
                  ('British Columbia, Canada', 'Alaska, US'),
                  ('British Columbia, Canada', 'Northwest Territories, Canada'),
                  ('Northwest Territories, Canada', 'Yukon, Canada'),
                  ('Yukon, Canada', 'Alaska, US')])

In [0]:
for node in comb_nodes:
  if 'Washington' in node or 'Oregon' in node or 'Idaho' in node or 'WA' in node:
    print(node)

Clark County, WA, US
Grant County, WA, US
Idaho, US
Jefferson County, WA, US
King County, WA, US
Kittitas County, WA, US
Oregon, US
Pierce County, WA, US
Seattle, WA, US
Snohomish County, WA, US
Spokane County, WA, US
Unassigned Location, WA, US
Washington, US
Washington County, OR, US
Washington, D.C., US
